# Behind the pipeline

## Preprocessing with a tokenizer

In [1]:
from transformers import AutoTokenizer

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [2]:
raw_inputs = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="tf")
print(inputs)

2024-04-05 14:46:54.909562: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-05 14:46:54.909802: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-05 14:46:55.186801: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


{'input_ids': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[  101,  1045,  1005,  2310,  2042,  3403,  2005,  1037, 17662,
        12172,  2607,  2026,  2878,  2166,  1012,   102],
       [  101,  1045,  5223,  2023,  2061,  2172,   999,   102,     0,
            0,     0,     0,     0,     0,     0,     0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 16), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)>}


## Going through the model

In [3]:
from transformers import TFAutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModel.from_pretrained(checkpoint)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertModel: ['pre_classifier.bias', 'classifier.bias', 'classifier.weight', 'pre_classifier.weight']
- This IS expected if you are initializing TFDistilBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFDistilBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


### A high-dimensional vector?

In [4]:
outputs = model(inputs)
print(outputs.last_hidden_state.shape)

(2, 16, 768)


In [5]:
outputs[0]

<tf.Tensor: shape=(2, 16, 768), dtype=float32, numpy=
array([[[-0.17977984,  0.23332864,  0.6320985 , ..., -0.30166635,
          0.50081986,  0.14814362],
        [ 0.2757769 ,  0.64971244,  0.3199769 , ..., -0.07599559,
          0.5136171 ,  0.13292244],
        [ 0.9045854 ,  0.09851371,  0.2949724 , ...,  0.3351949 ,
         -0.14074141, -0.6464028 ],
        ...,
        [ 0.14658947,  0.56606007,  0.32352793, ..., -0.33757478,
          0.5099775 , -0.05610842],
        [ 0.75000423,  0.04872622,  0.1738002 , ...,  0.46841496,
          0.00296654, -0.60837543],
        [ 0.05194443,  0.37294856,  0.52233255, ...,  0.35840544,
          0.65004265, -0.38829824]],

       [[-0.29370666,  0.7282561 , -0.14972687, ..., -0.11868069,
         -1.0226727 , -0.04215699],
        [-0.22063622,  0.938384  , -0.095125  , ..., -0.3643167 ,
         -0.66052216,  0.24069703],
        [-0.15360785,  0.89874977, -0.07276438, ..., -0.21891738,
         -0.8527601 ,  0.07099415],
        ...,


### Model heads: Making sense out of numbers

In [6]:
from transformers import TFAutoModelForSequenceClassification

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(inputs)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [7]:
outputs.logits.shape

TensorShape([2, 2])

### Postprocessing the output

In [8]:
print(outputs.logits)

tf.Tensor(
[[-1.5606961  1.6122813]
 [ 4.1692314 -3.3464477]], shape=(2, 2), dtype=float32)


In [9]:
import tensorflow as tf

predictions = tf.math.softmax(outputs.logits, axis=-1)
print(predictions)

tf.Tensor(
[[4.0195391e-02 9.5980465e-01]
 [9.9945587e-01 5.4418371e-04]], shape=(2, 2), dtype=float32)
